In [27]:
import yaml
import os
from langchain_anthropic import ChatAnthropic
from sentence_transformers import SentenceTransformer
from matching_functions import find_closest_documents

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
anthropic_key = config["anthropic_key"]
os.environ["ANTHROPIC_API_KEY"] = anthropic_key
chat = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")

<All keys matched successfully>


In [43]:
from wikipedia_functions import get_random_page, get_page_links, get_page_content, check_wikipedia_pages_existence

start_page, end_page = get_random_page(2)
end_page_full_content = get_page_content(end_page)
print(start_page)
print(end_page)

Zdeněk Simota
Vano Iantbelidze


In [44]:
start_links = get_page_links(start_page)
checked_links = check_wikipedia_pages_existence(start_links)
valid_links = [link for link in start_links if checked_links[link]]

In [45]:
from chain_functions import get_crawler_chain, get_summarize_chain, get_explain_links_chain, get_broad_links_chain

crawler_chain = get_crawler_chain(chat)
summarize_chain = get_summarize_chain(chat)
explain_links_chain = get_explain_links_chain(chat)
broad_links_chain = get_broad_links_chain(chat)

In [46]:
from answer_functions import answer_broad_links, answer_crawler, answer_explain_links, answer_summarize

end_page_content = answer_summarize(summarize_chain, end_page_full_content)
print(end_page_content)

Ivane "Vano" Iantbelidze is a Soviet and Georgian film, television, and stage actor born on January 14, 1954, in Samtredia, Georgian SSR. He graduated from the Shota Rustaveli Theatre and Film University in 1975 and has since played over 150 roles on the stage of the Telavi state drama theater. Iantbelidze has also appeared in several films, including "One-Two, Soldiers Were Going...", "Only Old Men Are Going to Battle", and "The Guard". Despite his extensive theatrical career, he has had limited appearances in cinema due to his high employment on the stage.


In [47]:
current_page = start_page
current_links = valid_links
forbidden_links = [start_page.strip()]
for i in range(10):
    if end_page in current_links:
        print(f"{end_page} in {current_page}'s links \nPage reached in {i+1} iterations!")
        break
    else:
        closest_links = find_closest_documents(model, current_links, end_page, end_page_content)
        print(closest_links)
        broad_links = answer_broad_links(broad_links_chain, current_links, end_page, end_page_content)
        print(broad_links)
        current_links = list(set([link for link in closest_links + broad_links if link in current_links]))
        reasonings = answer_explain_links(explain_links_chain, current_links, end_page, end_page_content)
        current_page = answer_crawler(crawler_chain, current_page, current_links, reasonings, end_page, end_page_content).strip()
        forbidden_links.append(current_page)
        found_links = get_page_links(current_page)
        checked_links = check_wikipedia_pages_existence(found_links)
        current_links = [link for link in found_links if checked_links[link] and link not in forbidden_links]        

['Prachatice', 'Orzeł Łódź', 'Plymouth Gladiators', 'Brovst Speedway Club', 'Peterborough Panthers']
['Premier League (speedway)', 'Elite League (speedway)', '2006 Speedway Grand Prix']
<reasoning>
Based on the provided information, the links that seem most relevant and potentially helpful in reaching the final page on Vano Iantbelidze are the ones related to speedway racing. The summary of the Vano Iantbelidze page mentions his acting career, but also notes that he has had limited appearances in cinema due to his high employment on the stage. By exploring the pages related to speedway racing, such as the 2006 Speedway Grand Prix, Premier League (speedway), and Elite League (speedway), I may be able to find connections or clues that could bridge the gap between Iantbelidze's acting career and the world of speedway racing, ultimately leading me to the final page.
</reasoning>

<output>
<output>2006 Speedway Grand Prix</output>
</output>
['Kasts Poudzuks', 'Latvijas Spīdveja Centrs', 'Wi